In [40]:
# import theano
# theano.config.device = 'gpu'
# theano.config.floatX = 'float32'

import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.layers.advanced_activations import LeakyReLU


bpd_crime_dataframe = pd.read_csv('BPD_Crime_sanitized.csv')
bpd_crime_dataframe = bpd_crime_dataframe.ix[~bpd_crime_dataframe.CrimeCode.isin(['8DO', '3LK', '6K', '8FV', '8CV', '3N', '8GV', '8EV', '3EO', '8I', '8CO', '8BV', '8GO', '3EK', '3LO', '3GO'])]

bpd_crime_dataframe =  bpd_crime_dataframe.sample(frac=1).reset_index(drop=True)

#X = pd.concat([pd.get_dummies(bpd_crime_dataframe['Weapon'], prefix = 'W'), pd.get_dummies(bpd_crime_dataframe['CrimeHour'], prefix = 'CH'), pd.get_dummies(bpd_crime_dataframe['CrimeDay'], prefix = 'CD')], axis = 1)
X = pd.concat([pd.get_dummies(bpd_crime_dataframe['Weapon'], prefix = 'W')], axis = 1)
Y = bpd_crime_dataframe['CrimeCode']
## Accuracy = 76.62%

#X_train = X.iloc[1:(len(X)/2)]
#X_test = X.iloc[(len(X)/2) + 1: len(X)]
#Y_train = Y.iloc[1:(len(Y)/2)]
#Y_test = Y.iloc[(len(Y)/2) + 1: len(Y)]

X_train = X[1:10000]
Y_train = Y[1:10000]
X_test = X[10000:20000]
Y_test = Y[10000:20000]

#print len(Y_train.unique()), ' & ', len(Y_test.unique())

In [41]:
encoder = LabelEncoder()
encoder.fit(Y_train)
encoded_Y_train = encoder.transform(Y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_train = np_utils.to_categorical(encoded_Y_train)

encoder.fit(Y_test)
encoded_Y_test = encoder.transform(Y_test)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_test = np_utils.to_categorical(encoded_Y_test)


# define baseline model
def baseline_model(input_size, output_size):
    # create model
    model = Sequential()
    model.add(Dense(input_size, input_dim=input_size, init='uniform', activation='relu'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dense(input_size, input_dim=input_size, init='uniform', activation='relu'))
    #model.add(LeakyReLU(alpha=0.001))
    model.add(Dense(output_size, init='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

build_fn = baseline_model(len(X_train.columns), len(Y.unique()))
build_fn.fit(X_train.values, dummy_y_train, batch_size=5, nb_epoch = 20, verbose=1, shuffle=True)
build_fn.evaluate(X_test.values, dummy_y_test, batch_size=5, verbose=1)

#estimator = KerasClassifier(build_fn=baseline_model, nb_epoch=15, batch_size=5, verbose=1)
#kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
#results = cross_val_score(estimator, X.values, dummy_y, cv=kfold)
#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

# X_predict = pd.concat([pd.get_dummies(test_crime_dataframe['Neighborhood'], prefix = 'NB'), pd.get_dummies(test_crime_dataframe['District'], prefix = 'DS'), pd.get_dummies(test_crime_dataframe['Inside/Outside'], prefix = 'IO'), pd.get_dummies(test_crime_dataframe['Weapon'], prefix = 'W'), pd.get_dummies(test_crime_dataframe['CrimeHour'], prefix = 'CH'), pd.get_dummies(test_crime_dataframe['CrimeDay'], prefix = 'CD')], axis = 1)
# Y = train_crime_dataframe['CrimeCode']

Epoch 1/20
9999/9999 [==============================] - 2s - loss: 1.9221 - acc: 0.4924     
Epoch 2/20
9999/9999 [==============================] - 2s - loss: 1.3724 - acc: 0.5359     
Epoch 3/20
9999/9999 [==============================] - 1s - loss: 1.2559 - acc: 0.6119     
Epoch 4/20
9999/9999 [==============================] - 1s - loss: 1.1710 - acc: 0.6119     
Epoch 5/20
9999/9999 [==============================] - 2s - loss: 1.1010 - acc: 0.6364     
Epoch 6/20
9999/9999 [==============================] - 1s - loss: 1.0460 - acc: 0.6839     
Epoch 7/20
9999/9999 [==============================] - 1s - loss: 0.9984 - acc: 0.7334     
Epoch 8/20
9999/9999 [==============================] - 1s - loss: 0.9553 - acc: 0.7612     
Epoch 9/20
9999/9999 [==============================] - 1s - loss: 0.9192 - acc: 0.7616     
Epoch 10/20
9999/9999 [==============================] - 1s - loss: 0.8753 - acc: 0.7616     
Epoch 11/20
9999/9999 [==============================] - 1s - loss: 0

[0.73041131758317346, 0.76629999999999998]